In [2]:
#This code is used to free GPU cache when necessary and have no relation to below code.
!pip install GPUtil

from GPUtil import showUtilization as gpu_usage
gpu_usage()                             
import torch
torch.cuda.empty_cache()
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)


| ID | GPU | MEM |
------------------
|  0 |  0% |  1% |
|  1 |  0% |  0% |


<weakproxy at 0x7923337f1260 to Device at 0x79233b60df10>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [4]:
import wandb
import os

os.environ['WANDB_API_KEY'] = '1ffc33d77af0fd022201ec32b81cd0e92cd75821'
wandb.login()

wandb: Currently logged in as: alandandoor (alandandoor-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
sweep_config = {
    'method': 'bayes', 
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'dropout': {
            'values': [0.2, 0.3]
        },
        'lr': {
            'values': [0.0001, 0.001]
        },
        'activation': {
            'values': ['SiLU', 'Mish']
        },
        'optimizer': {
            'values': ['adam', 'nadam']
        },
        'batch_norm':{
            'values': ['true','false']
        },
        'data_aug': {
            'values': ['true','false']
        },
        'batch_size': {
            'values': [64, 128]
        },
        'num_dense':{
            'values': [64, 128]
        }
    }
}


sweep_id = wandb.sweep(sweep=sweep_config, project='DL_A2')

Create sweep with ID: 7r7nif34
Sweep URL: https://wandb.ai/alandandoor-iit-madras/DL_A2/sweeps/7r7nif34


In [6]:
class ResNet(nn.Module):
    def __init__(self, config, num_classes=10):
        super(CNN, self).__init__()
        self.config =config
        self.device =torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model =models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

        for param in self.model.parameters():
            param.requires_grad=False        

        modules =[]
        if config.dropout>0:
            modules.append(nn.Dropout(config.dropout))
        modules.append(nn.Linear(self.model.fc.in_features, config.num_dense))

        if config.batch_norm=="true":
            modules.append(nn.BatchNorm1d(config.num_dense))

        modules.append(self.get_activation(config.activation))
        modules.append(nn.Linear(config.num_dense, num_classes))

        self.model.fc =nn.Sequential(*modules)

        for param in self.model.fc.parameters():
            param.requires_grad =True

        self.model =self.model.to(self.device)

        self.criterion =nn.CrossEntropyLoss()
        if config.optimizer=="adam":
            self.optimizer =optim.Adam(self.model.parameters(), lr=config.lr)
        elif config.optimizer=="nadam":
            self.optimizer =optim.NAdam(self.model.parameters(), lr=config.lr)

        self.train_loader, self.val_loader=self.get_data_loaders()

    def get_activation(self, name):
        if name=="ReLU":
            return nn.ReLU()
        elif name=="GELU":
            return nn.GELU()
        elif name=="SiLU":
            return nn.SiLU()
        else:
            return nn.Mish()


    def get_data_loaders(self):
        mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

        train_transforms = [transforms.Resize((224, 224))]
        if self.config.data_aug=="true":
            train_transforms += [transforms.RandomHorizontalFlip(),transforms.RandomRotation(15),]
        train_transforms += [transforms.ToTensor(), transforms.Normalize(mean, std)]

        val_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
        ])

        train_dataset = datasets.ImageFolder("/kaggle/input/dl-assignment-2/inaturalist_12K/train",transform=transforms.Compose(train_transforms),)
        val_dataset = datasets.ImageFolder("/kaggle/input/dl-assignment-2/inaturalist_12K/val",transform=val_transforms,)

        train_loader = DataLoader(train_dataset, batch_size=self.config.batch_size, shuffle=True, num_workers=2)
        val_loader = DataLoader(val_dataset, batch_size=self.config.batch_size, shuffle=False, num_workers=2)

        return train_loader, val_loader

    def train_model(self, num_epochs=10):
        for epoch in range(num_epochs):
            self.model.train()
            running_loss, correct, total = 0, 0, 0
            for images, labels in self.train_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

            train_loss = running_loss / total
            train_acc = correct / total

            val_loss, val_acc = self.validate()

            train_acc*=100
            val_acc*=100

            wandb.log({"train_loss": train_loss,"train_acc": train_acc,"val_loss": val_loss,"val_acc": val_acc,})

            print(f"Epoch {epoch+1}: Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Val Loss: {val_loss:.4f}")

        torch.save(self.model.state_dict(), "/kaggle/working/finetuned_resnet50.pth")

    def validate(self):
        self.model.eval()
        val_loss = 0
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for images, labels in self.val_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        avg_val_loss = val_loss / val_total
        val_acc = val_correct / val_total
        return avg_val_loss, val_acc


In [ ]:
def main():
    with wandb.init() as run:
        config = wandb.config

        bn = config.batch_norm
        da = config.data_aug

        wandb.run.name = (
            f"{config.activation}-{config.optimizer}-bn_{bn}-da_{da}-do_{config.dropout}"
            f"-bs_{config.batch_size}-lr_{config.lr}-fc_{config.num_dense}"
        )

        model = CNN(config, num_classes=10)
        model.train_model()

# Replace with your actual sweep ID
wandb.agent('m2v2xftu', function=main)
wandb.finish()


wandb: Agent Starting Run: ukrqv79e with config:
wandb: 	activation: SiLU
wandb: 	batch_norm: false
wandb: 	batch_size: 128
wandb: 	data_aug: true
wandb: 	dropout: 0.3
wandb: 	lr: 0.0001
wandb: 	num_dense: 128
wandb: 	optimizer: adam
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 217MB/s]


Epoch 1: Train Acc: 48.3848, Val Acc: 64.2000, Val Loss: 1.3650
Epoch 2: Train Acc: 64.1864, Val Acc: 70.3500, Val Loss: 1.0528
Epoch 3: Train Acc: 67.3167, Val Acc: 71.2500, Val Loss: 0.9216
Epoch 4: Train Acc: 68.5069, Val Acc: 72.1500, Val Loss: 0.8837
Epoch 5: Train Acc: 69.9870, Val Acc: 72.2500, Val Loss: 0.8544
Epoch 6: Train Acc: 70.2570, Val Acc: 71.6500, Val Loss: 0.8388
Epoch 7: Train Acc: 70.7871, Val Acc: 73.8500, Val Loss: 0.8034
Epoch 8: Train Acc: 71.3971, Val Acc: 73.4500, Val Loss: 0.7979
Epoch 9: Train Acc: 71.7072, Val Acc: 73.1500, Val Loss: 0.7988
Epoch 10: Train Acc: 72.1372, Val Acc: 72.8500, Val Loss: 0.7967


train_acc,▁▆▇▇▇▇████
train_loss,█▄▂▂▂▁▁▁▁▁
val_acc,▁▅▆▇▇▆██▇▇
val_loss,█▄▃▂▂▂▁▁▁▁
train_acc,72.13721
train_loss,0.85263
val_acc,72.85
val_loss,0.7967


wandb: Agent Starting Run: ii5ym9z8 with config:
wandb: 	activation: SiLU
wandb: 	batch_norm: false
wandb: 	batch_size: 64
wandb: 	data_aug: false
wandb: 	dropout: 0.3
wandb: 	lr: 0.001
wandb: 	num_dense: 64
wandb: 	optimizer: adam


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: nmmwl9wd with config:
wandb: 	activation: Mish
wandb: 	batch_norm: false
wandb: 	batch_size: 64
wandb: 	data_aug: true
wandb: 	dropout: 0.2
wandb: 	lr: 0.0001
wandb: 	num_dense: 64
wandb: 	optimizer: nadam


Epoch 1: Train Acc: 47.6048, Val Acc: 65.9000, Val Loss: 1.3407
Epoch 2: Train Acc: 65.6166, Val Acc: 70.0000, Val Loss: 1.0521
Epoch 3: Train Acc: 67.8868, Val Acc: 70.2000, Val Loss: 0.9606
Epoch 4: Train Acc: 69.4169, Val Acc: 71.8000, Val Loss: 0.8797
Epoch 5: Train Acc: 70.1270, Val Acc: 71.5500, Val Loss: 0.8603
Epoch 6: Train Acc: 71.1271, Val Acc: 71.8000, Val Loss: 0.8435
Epoch 7: Train Acc: 70.7671, Val Acc: 72.6000, Val Loss: 0.8216
Epoch 8: Train Acc: 71.1371, Val Acc: 72.5500, Val Loss: 0.8011
Epoch 9: Train Acc: 71.7272, Val Acc: 72.9500, Val Loss: 0.7959
Epoch 10: Train Acc: 72.3872, Val Acc: 72.5500, Val Loss: 0.7870


train_acc,▁▆▇▇▇█████
train_loss,█▄▃▂▂▁▁▁▁▁
val_acc,▁▅▅▇▇▇████
val_loss,█▄▃▂▂▂▁▁▁▁
train_acc,72.38724
train_loss,0.8407
val_acc,72.55
val_loss,0.78701


wandb: Agent Starting Run: 1jsq9mhp with config:
wandb: 	activation: Mish
wandb: 	batch_norm: false
wandb: 	batch_size: 64
wandb: 	data_aug: true
wandb: 	dropout: 0.3
wandb: 	lr: 0.0001
wandb: 	num_dense: 128
wandb: 	optimizer: adam


Epoch 1: Train Acc: 51.2151, Val Acc: 68.0500, Val Loss: 1.1543
Epoch 2: Train Acc: 65.6266, Val Acc: 71.1000, Val Loss: 0.9444
Epoch 3: Train Acc: 67.9568, Val Acc: 71.8000, Val Loss: 0.8854
Epoch 4: Train Acc: 68.5169, Val Acc: 72.9500, Val Loss: 0.8313
Epoch 5: Train Acc: 69.6270, Val Acc: 73.2000, Val Loss: 0.8188
